<a href="https://colab.research.google.com/github/SciEcon/SRS2023_MEV_Jay/blob/main/code/MEVblocker_data_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import requests
from requests import get, post

In [13]:
import pandas as pd
import numpy as np


In [14]:
BASE_URL = "https://api.dune.com/api/v1/"

def make_api_url(module, action, ID):
    """
    We shall use this function to generate a URL to call the API.
    """

    url = BASE_URL + module + "/" + ID + "/" + action

    return url

In [19]:
def execute_query(query_id, engine="medium"):
    """
    Takes in the query ID and engine size.
    Specifying the engine size will change how quickly your query runs.
    The default is "medium" which spends 10 credits, while "large" spends 20 credits.
    Calls the API to execute the query.
    Returns the execution ID of the instance which is executing the query.
    """

    url = make_api_url("query", "execute", query_id)
    params = {
        "performance": engine,
    }
    response = post(url, headers=HEADER, params=params)
    execution_id = response.json()['execution_id']

    return execution_id


def get_query_status(execution_id):
    """
    Takes in an execution ID.
    Fetches the status of query execution using the API
    Returns the status response object
    """

    url = make_api_url("execution", "status", execution_id)
    response = get(url, headers=HEADER)

    return response


def get_query_results(execution_id):
    """
    Takes in an execution ID.
    Fetches the results returned from the query using the API
    Returns the results response object
    """

    url = make_api_url("execution", "results", execution_id)
    response = get(url, headers=HEADER)

    return response


def cancel_query_execution(execution_id):
    """
    Takes in an execution ID.
    Cancels the ongoing execution of the query.
    Returns the response object.
    """

    url = make_api_url("execution", "cancel", execution_id)
    response = get(url, headers=HEADER)

    return response


In [22]:
url='https://api.dune.com/api/v1/query/2441751/results/csv?api_key=<Your_API>'

In [36]:
API_KEY = 'Your_API'
HEADER = {"x-dune-api-key" : API_KEY}

In [37]:
response = get(url,headers = HEADER)

In [38]:
response


<Response [200]>

In [46]:
execution_id = execute_query("2441751","large")


In [47]:
execution_id

'01H6TG9W2EK0RCVWY1R8Q65BRS'

In [48]:
response = get_query_status(execution_id)

In [49]:
response

<Response [200]>

In [55]:
response = get_query_results(execution_id)
print(response.json()['result'])

{'rows': [{'day': '2023-07-08 00:00:00.000 UTC', 'num_evm_tx': 1014086, 'num_mevblocker_tx': 56001, 'tx_proportion': 5.5223127032618535}, {'day': '2023-08-01 00:00:00.000 UTC', 'num_evm_tx': 1068108, 'num_mevblocker_tx': 55405, 'tx_proportion': 5.187209533118374}, {'day': '2023-07-30 00:00:00.000 UTC', 'num_evm_tx': 981157, 'num_mevblocker_tx': 38574, 'tx_proportion': 3.931480894494969}, {'day': '2023-06-21 00:00:00.000 UTC', 'num_evm_tx': 1091714, 'num_mevblocker_tx': 41137, 'tx_proportion': 3.768111428451041}, {'day': '2023-06-14 00:00:00.000 UTC', 'num_evm_tx': 1023424, 'num_mevblocker_tx': 36768, 'tx_proportion': 3.592645863298105}, {'day': '2023-06-04 00:00:00.000 UTC', 'num_evm_tx': 962454, 'num_mevblocker_tx': 39723, 'tx_proportion': 4.127262186036943}, {'day': '2023-06-20 00:00:00.000 UTC', 'num_evm_tx': 1072045, 'num_mevblocker_tx': 41141, 'tx_proportion': 3.837618756675326}, {'day': '2023-07-15 00:00:00.000 UTC', 'num_evm_tx': 950700, 'num_mevblocker_tx': 49767, 'tx_proportio

In [56]:
data = pd.DataFrame(response.json()['result']['rows'])

In [57]:
data

,day,num_evm_tx,num_mevblocker_tx,tx_proportion
0,2023-07-08 00:00:00.000 UTC,1014086,56001,5.522313
1,2023-08-01 00:00:00.000 UTC,1068108,55405,5.187210
2,2023-07-30 00:00:00.000 UTC,981157,38574,3.931481
3,2023-06-21 00:00:00.000 UTC,1091714,41137,3.768111
4,2023-06-14 00:00:00.000 UTC,1023424,36768,3.592646
...,...,...,...,...
96,2023-04-27 00:00:00.000 UTC,939262,9620,1.024208
97,2023-07-14 00:00:00.000 UTC,1110332,50164,4.517928
98,2023-05-26 00:00:00.000 UTC,1059510,49869,4.706798
99,2023-06-27 00:00:00.000 UTC,1067370,39498,3.700497


In [58]:
data.to_csv('MEVblocker.csv',index=False)

In [59]:
response = cancel_query_execution(execution_id)